In [ ]:
from ast import fix_missing_locations
import os 
import tarfile
import urllib
import urllib.request
import pandas as pd
import numpy as np
from zlib import crc32
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from pandas.plotting import scatter_matrix
from scipy import stats
import matplotlib.pyplot as plt
import joblib


DL_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/" # root for datasets
H_PATH = os.path.join("datasets", "housing") 
H_URL = DL_ROOT + "datasets/housing/housing.tgz" # full url for the housing csv


# when calling fetch_housing_data(), it creates a `datasets/housing` directory, downloads the tar file and extract housing.csv file in the dir
def fetch_housing_data(housing_url = H_URL, housing_path=H_PATH):
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, "housing.tgz")    
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

# load csv file
def load_housing_data(housing_path=H_PATH):
    csv_path = os.path.join(housing_path ,"housing.csv")
    return pd.read_csv(csv_path)

fetch_housing_data()



#########################################################################################
#                                                                                       #
#                                                                                       #
#                                    DATA EXPLORATION                                   #
#                                                                                       #
#                                                                                       #
#########################################################################################


housing = load_housing_data()
# shows top five rowq usisng DataFrame's head() method
housing.head()
# the info() method is useful to get a quick desc of the data. in particular the total number of rows, each attribute's type, and the number of nonnull values
housing.info()
# value_counts() counts the number of value who have a specific label
housing["ocean_proximity"].value_counts()
# describe() method shows a summary of the numeriacal attributes
housing.describe()
#
# it also exists count(), mean(), min(), max(). They are self-explanatory 
#
#
# TO MAKES SOME FANCY GRAPH:
# 
# hist() method works by using matplotlib, which do some magic that needs the first line to be `*matplotlib inline`
# 
# 
# %matplotlib inline # note that this works only in jupyter notebook
# import matplotlib.pyplot as plt
# housing.hist(bins=50, figsize=(20,15))
# plt.show() 


# we have to create two dataset: testing and training. here is an implementation:
###### NOTE
# this is not the best implementation, because of risk in case of refresh of datasets, and restart of computing for the ML 
# there's a better implementation for that using hash below this commented-code section  
#
# def split_train_test(data, test_ratio):
#     # shuffle datasets
#     shuffled_indices = np.random.permutation(len(data))
#     test_set_size = int(len(data)*test_ratio)
#     # takes a test dataset
#     test_indices = shuffled_indices[:test_set_size]
#     # takes a training dataset
#     train_indices = shuffled_indices[test_set_size:]
#     return data.iloc[train_indices], data.iloc[test_indices]
#
# create test & training dataset
# 
# train_set, test_set = split_train_test(housing, 0.2)
# print(len(train_set))
# print(len(test_set))

# use each instance's identifier to decide wether or not it should go in the test set (assuming instances have a unique and immutable identifier)
# compute a hash of each instance's identifier and put that instance in the test set if the hash is lower than or equal to 20% of the maximum hash value
# this ensure that the test set will remain consistent across multiple runs, even if you refresh the dataset
def test_set_check(identifier, test_ratio):
    return crc32(np.int64(identifier)) & 0xffffffff < test_ratio * 2**32
# 
def split_train_test_by_id(data, test_ratio, id_column):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio))
    return data.loc[~in_test_set], data.loc[in_test_set]

housing_with_id = housing.reset_index() # adds and `index` column
# build a unique identifier. here : district's latitude and longitude are guaranteed to be stable for a few million years
# so i combine them to create an ID like so: 
housing_with_id["id"] = housing["longitude"] * 1000 + housing["latitude"]
train_set, test_set = split_train_test_by_id(housing_with_id, 0.2, "index")
print("end of creation of training and testing dataset")

housing["income_cat"] = pd.cut(housing["median_income"],
                              bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                              labels=[1,2,3,4,5])

housing["income_cat"].hist()

# stratified sampling based on the income category. we're using Scikit-Learn's `StratifiedShuffleSplit`:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]
# verify if it worked: here we're looking at the income category proportions in the test set
strat_test_set["income_cat"].value_counts() / len(strat_test_set)

# TO ONLY 

# because i like fancy graphic, here's how to do: 
# (using pandas for that)
# kind allows to choose different plot type (for more info https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.DataFrame.plot.html)
# x and y are the label for the x and y axis
# alpha=0.1 makes it much easier to visualize the places where there's a high density of data points
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1)

# for a more precise / informative plot:
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1,
             s=housing["population"]/100, label="population",figsize=(10,7),
             c="median_house_value", cmap=plt.get_cmap("jet"), colorbar=True,
             )
# show the graphs
plt.legend()

# compute the Standard Correlation Coefficient (also called Pearson's r) between every paire of attributes using the corrr() method
# this works because the dataset is not too large
corr_matrix = housing.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

# another method to check for correlation between attributes is to use the pandas `scatter_matrix()` function, which plots every numerical against every other
# numerical attribute
attributes = ["median_house_value", "median_income",  "total_rooms", "housing_median_age"]
scatter_matrix(housing[attributes], figsize=(12,8))


#########################################################################################
#                                                                                       #
#                                                                                       #
#                                      DATA CLEANING                                    #
#                                                                                       #
#                                                                                       #
#########################################################################################

# clear training set (by copying strat_train_set)
# separate the predictors and the labels, since we don't necessarily want to apply the same transformations to the predictors and the target values
# note that here, `drop()` creates a copy of the data and does not affect strat_train_set
housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

housing.dropna(subset=["total_bedrooms"])

# Scikit-Learn provides a handy class to take care of missing values: SimpleImputer
# First, you need to create a SimpleImputer() instance, specifying that you want to replace each attribute's missing values with the median of that attribute
imputer = SimpleImputer(strategy="median")
# since the median can only be computed on numerical attributes, you need yo create a copy of the data without the text attribute ocean_proximity
housing_num = housing.drop("ocean_proximity", axis=1)
# Now you can fit the imputer instance to the training data using the fit() method
imputer.fit(housing_num)

# text attributes
housing_cat = housing[["ocean_proximity"]]
# shows it's 10 first attributes
housing_cat.head(10)
# this is a categorical attribute, so we need to convert it to numbers
ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)
housing_cat_encoded[:10]
# you can get the list of categories by using the categories_ instance variable
# it's a list containing a 1D array of categories for each categorical attribute
ordinal_encoder.categories_ 
# create a binary attribute per category: one attribute equal to 1 when the category is "<1H OCEAN" (and 0 otherwise), and so on.
# This is called one-hot encoding, because only one attribute will be equal to one
# The new attributes are sometimes called dummy attributes. Scikit-Learn provides a `OneHotEncoder()` class to convert categorical values into onhot vectors
cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot.toarray()



#########################################################################################
#                                                                                       #
#                                                                                       #
#                                  CUSTOM TRANSFORMERS                                  #
#                                                                                       #
#                                                                                       #
#########################################################################################

# you may need to wrute your own transformers for tasks such as custom cleanup operations or combining specific attributes
# all you need to do is create a class and implement three methods: fit(), transform() and fit_transform()
# you can get the last one for free simply by adding TransformerMixin as a base class
# if you add BaseEstimator as a base class (and avoid *args and **kargs in your constructor), you will also get two extra methods (get_params() and set_params())
# that will be useful for automatic hyperparameter tuning
#
# The following is a small transformer class that adds the combined attributes

rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True): # no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
    
    def fit(self, X, y=None):
        return self # nothing else to do

    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, population_ix] / X[:, households_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]
        
attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=True)
housing_extra_attribs = attr_adder.transform(housing.values)


#########################################################################################
#                                                                                       #
#                                                                                       #
#                               TRANSFORMATION PIPELINE                                 #
#                                                                                       #
#                                                                                       #
#########################################################################################


# There are many data transformation steps that need to be executed in the right order
# Scikit-Learn provides the Pipeline class to help with such sequences of transformatioins
# The Pipeline constructor takes a list of name/estimator pairs defining a sequence of steps.

num_pipeline = Pipeline([('imputer', SimpleImputer(strategy="median")),
                        ('attribs_adder', CombinedAttributesAdder()),
                        ('std_scaler', StandardScaler()),
                        ])

housing_num_tr = num_pipeline.fit_transform(housing_num)

# We get the list of numerical column names and the list of categorical column names, and then we construct ColumnTransformer
# The constructor requires a list of tuples, where each tuple contains a name, a transformer, and a list of names (or indices) of columns that the transformer
# should be applied to.
# we specify that the numerical columns should be transformed using the num_pipeline that we defined earlier, and the categorical columns should be transformed
# using a OneHotEncoder
# Finally we apply this ColumnTransformer to the housing data: it applies each transformer to the appropriate columns and concatenates the outputs along the 
# second axis
##### NOTE
# OneHotEncoder returns a sparse matrix, while the num_pipeline returns a dense matrix. 
num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]
full_pipeline = ColumnTransformer([("num", num_pipeline, num_attribs),
                                  ("cat", OneHotEncoder(), cat_attribs),
                                  ])

housing_prepared = full_pipeline.fit_transform(housing)


#########################################################################################
#                                                                                       #
#                                                                                       #
#                              SELECT AND TRAIN A MODEL                                 #
#                                                                                       #
#                                                                                       #
#########################################################################################


# The first (and probably easier one) is the Linear Regression
# Let's first train a Linear Regression model, like we did previously
lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)
# Done! We now have a Linear Regression model
# Let's try it out on a few instances from the training set
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)
print("Predictions:", lin_reg.predict(some_data_prepared))
print("Labels:", list(some_labels))
# It works, although the predictions are not exactly accurate
# Let's measure this regression model's RMSE on the whole training set using Scikit-Learn's mean_squared_error()
# What is RMSE ?
# RMSE is a performance measure. It stands for Root Mean Square Error. It gives and idea of how much error the system typically makes in its prediction
# with a higher weight for large errors.

housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

# This is better than nothing, but clearly not a great score: most districts' median_housing_values range between $120,000 abd $256,000
# so a typical error prediction error of ~$68,000 is not very satisfying. This model is underfitting the training data
# This either means that we do not provides enough data, or that the model is not powerful enough
# We can fix underfitting by: selecting a more powerful model, to feed the training algorithm with better features, or to reduce the constraints on the model
# Let's try a more complex model!

tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)
# Now that it's trained, let's evaluate it on the training set:
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
print(tree_rmse)

# Whaaaaaat?! No error at all? So this model is absolutely perfect? It's possible that the model has badly ofervit the data.
# How can you be sure?
# You need to not touch the test dataset until you are ready to launch a model you're confident about, so you need to use part of the training set for training

# Introducing Cross-Validation
# One way to evaluate the Decision Tree model would be to use the train_test_split() function to split the training set into smaller training set and 
# a validation set. 
# It requires a little bit of work, but nothing too difficult
# 
# A great alternative is to use Scikit-Learn's K-fold cross-validation feature

scores = cross_val_score(tree_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)
# now let's look at the results:
def display_scores(scores):
    print("Scores: ", scores)
    print("Mean: ", scores.mean())
    print("Standard deviation: ", scores.std())

display_scores(tree_rmse_scores)

# Now it doesn't look as good as it did earlier. In fact it seems to perform worse than the Linear Regression
# let's compute the same for the Linear Regression
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

# Let's try one last model: RandomForestRegressor
# Random Forests work by training many Decision Trees on random subsets of the features, then averaging out their predictions
# Building a model on top of other models is called Ensemble Learning, and it is often a great way to push ML algorithms even further

forest_reg = RandomForestRegressor()
forest_reg.fit(housing_prepared, housing_labels)

forest_scores = cross_val_score(forest_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

# That's better, however the model is still overfitting


#########################################################################################
#                                                                                       #
#                                                                                       #
#                                  FINE TUNE YOUR MODEL                                 #
#                                                                                       #
#                                                                                       #
#########################################################################################

# You should use Scikit-Learn's GridSearchCV to search for you the best params for your model.

param_grid = [{'n_estimators':[3,10,30],'max_features':[2,4,6,8]},
              {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2,3,4]},
              ]

forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring="neg_mean_squared_error", return_train_score=True)

grid_search.fit(housing_prepared, housing_labels)
# show the best params
grid_search.best_params_
# {'max_features': 8, 'n_estimators': 30}
grid_search.best_estimator_
# RandomForestRegressor(max_features=8, n_estimators=30)
#The evalutation score are available in case you want to see all of them
#
# cvres = grid_search.cv_results_
# for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
#     print(np.sqrt(-mean_score),params)
#
# Here we obtain the best solution by setting max_features hyperparameter to 8 and n_estimators to 30

# You could also use RandomizedSearchCV instead of GridSearchCV
# it has some benefits when the hyperparameter search soace is large

# Let's analyze the best models and their errors
# to do so, we need to display the importance scores next to their corresponding attribute names:
feature_importances = grid_search.best_estimator_.feature_importances_
extra_attribs = ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_room"]
cat_encode = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attribtues = num_attribs + extra_attribs + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)

# With those info we now know we could drop some less useful features


#########################################################################################
#                                                                                       #
#                                                                                       #
#                                  FINE TUNE YOUR MODEL                                 #
#                                                                                       #
#                                                                                       #
#########################################################################################

# We're now going to evaluate the system on the test set

final_model = grid_search.best_estimator_

X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

X_test_prepared = full_pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
display_scores(final_rmse)

# Although it is better, how can you be sure of that?
# well you can compute a 95% confidence interval for the generalization error using scipy.stats.t.interval() 

confidence = 0.95
squared_errors = (final_predictions - y_test) ** 2
np.sqrt(stats.t.interval(confidence, len(squared_errors)-1, loc=squared_errors.mean(), scale=stats.sem(squared_errors)))

# now that the model is done and you can use it, just save it with joblib

joblib.dump(final_model, "my_model.pkl") # final_model is my fine-tuned model
# you can then load it later on by doing the following
my_model_loaded = joblib.load("my_model.pkl")



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB
end of creation of training and testing dataset
Predictions: [ 87112.22868401 311287.67895144 149138.7569972  181605.04417126
 242876.06079414]
Labels: [72100.0, 279600.0, 82700.0, 112500.0, 238300.0]
68740.3505914331
0.0
Scores:  [71441.95984286 6897